# CIFAR-100 Classification Using Fine-Tuned ResNet-50
This notebook uses a pretrained **ResNet-50** model and applies **transfer learning** to classify images in the CIFAR-100 dataset. We use data augmentation, layer freezing, learning rate scheduling, and visualize performance.

In [ ]:
!pip install -q torchmetrics lightning-utilities

## Import Required Libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import torchmetrics
from torchmetrics import Accuracy, ConfusionMatrix
from torchvision.utils import make_grid

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

## Load CIFAR-100 Dataset with Augmentation

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor()
])

transform_test = transforms.Compose([transforms.ToTensor()])

train_dataset = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)

train_set, val_set = torch.utils.data.random_split(train_dataset, [40000, 10000])
train_loader = torch.utils.data.DataLoader(train_set, batch_size=64, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_set, batch_size=64, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

classes = train_dataset.classes

## Load and Fine-Tune Pretrained ResNet-50

In [ ]:
model = models.resnet50(weights='IMAGENET1K_V1')

for param in model.parameters():
    param.requires_grad = False

for param in model.layer4.parameters():
    param.requires_grad = True
for param in model.fc.parameters():
    param.requires_grad = True

model.fc = nn.Sequential(
    nn.Dropout(0.4),
    nn.Linear(model.fc.in_features, 100)
)

model = model.to(device)

## Define Loss, Optimizer, and Scheduler

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

## Training and Validation Functions

In [ ]:
def train_one_epoch():
    model.train()
    acc = Accuracy(task='multiclass', num_classes=100).to(device)
    total_loss = 0
    for X, Y in train_loader:
        X, Y = X.to(device), Y.to(device)
        optimizer.zero_grad()
        preds = model(X)
        loss = criterion(preds, Y)
        loss.backward()
        optimizer.step()
        acc.update(preds.argmax(dim=1), Y)
        total_loss += loss.item() * X.size(0)
    return total_loss / len(train_loader.dataset), acc.compute().item()

def validate(loader):
    model.eval()
    acc = Accuracy(task='multiclass', num_classes=100).to(device)
    total_loss = 0
    with torch.no_grad():
        for X, Y in loader:
            X, Y = X.to(device), Y.to(device)
            preds = model(X)
            loss = criterion(preds, Y)
            acc.update(preds.argmax(dim=1), Y)
            total_loss += loss.item() * X.size(0)
    return total_loss / len(loader.dataset), acc.compute().item()

## Training Loop

In [ ]:
history = pd.DataFrame()
epochs = 20

for epoch in range(epochs):
    train_loss, train_acc = train_one_epoch()
    val_loss, val_acc = validate(val_loader)
    scheduler.step()
    print(f"Epoch {epoch+1}: Train Acc={train_acc:.4f}, Val Acc={val_acc:.4f}")
    history = pd.concat([history, pd.DataFrame({
        'epoch': [epoch],
        'train_loss': [train_loss],
        'train_acc': [train_acc],
        'val_loss': [val_loss],
        'val_acc': [val_acc]
    })], ignore_index=True)

## Visualize Training Metrics

In [ ]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(history['epoch'], history['train_loss'], label='Train Loss')
plt.plot(history['epoch'], history['val_loss'], label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.subplot(1,2,2)
plt.plot(history['epoch'], history['train_acc'], label='Train Accuracy')
plt.plot(history['epoch'], history['val_acc'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## Final Evaluation and Confusion Matrix

In [ ]:
model.eval()
test_acc = Accuracy(task='multiclass', num_classes=100)
conf_matrix = ConfusionMatrix(task='multiclass', num_classes=100)

with torch.no_grad():
    for X, Y in test_loader:
        preds = model(X.to(device)).argmax(dim=1)
        test_acc.update(preds.cpu(), Y)
        conf_matrix.update(preds.cpu(), Y)

print("Test Accuracy:", test_acc.compute().item())

In [ ]:
plt.figure(figsize=(12,10))
sns.heatmap(conf_matrix.compute().numpy(), cmap="Blues")
plt.title("Confusion Matrix Heatmap")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

## Sample Predictions for Qualitative Analysis

In [ ]:
fig, axes = plt.subplots(2, 4, figsize=(12,6))
model.eval()
with torch.no_grad():
    for i in range(8):
        img, label = test_dataset[i]
        pred = model(img.unsqueeze(0).to(device)).argmax(dim=1).item()
        ax = axes[i//4, i%4]
        ax.imshow(img.permute(1,2,0))
        ax.set_title(f"True: {classes[label]}\nPred: {classes[pred]}")
        ax.axis('off')
plt.tight_layout()
plt.show()